<a href="https://colab.research.google.com/github/pcsilcan/pcd/blob/master/20202/pcd_20202_0701_openmp_ftw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenMP

In [5]:
%%writefile 1.cpp
#include <stdio.h>
#include <omp.h>

int main() {
#pragma omp parallel
{
    int id = omp_get_thread_num();
    printf("Hello(%d), ", id);
    printf("World(%d)!\n", id);
}

    return 0;
}

Overwriting 1.cpp


In [12]:
!g++ -std=c++17 -fopenmp 1.cpp && ./a.out

Hello(0), World(0)!
Hello(1), World(1)!


# Calc $\pi$

## Serial

In [58]:
%%writefile pi0.cpp
#include <stdio.h>
#include <omp.h>

double calcPi(int num_steps) {
    double step;
    double x, pi, sum = .0;
    step = 1. / (double)num_steps;
    for (int i = 0; i < num_steps; ++i) {
        x = (i + .5) * step;
        sum += 4. / (1 + x*x);
    }
    pi = sum / num_steps;
    return pi;
}

int main() {
    double pi;
    double ti = omp_get_wtime();
    pi = calcPi(100000);
    double et = omp_get_wtime() - ti;
    printf("Pi = %lf time = %lfs\n", pi, et);
    return 0;
}

Writing pi0.cpp


## Parallel v1

In [59]:
%%writefile pi1.cpp
#include <stdio.h>
#include <omp.h>

#define NUM_THREADS 2

double calcPi(int num_steps) {
    double step;
    int nthreads;
    double pi, sum[NUM_THREADS];
    step = 1. / (double)num_steps;
    omp_set_num_threads(NUM_THREADS);
#pragma omp parallel
{
    double  x;
    int     nthrds = omp_get_num_threads();
    int     id = omp_get_thread_num();
    if (id == 0) nthreads = nthrds;
    sum[id] = 0.0;
    for (int i = id; i < num_steps; i+=nthrds) {
        x = (i + .5) * step;
        sum[id] += 4. / (1 + x*x);
    }
}
    for (int i = 0; i < nthreads; ++i) {
        pi += sum[i];
    }
    return pi / num_steps;
}

int main() {
    double pi;
    double ti = omp_get_wtime();
    pi = calcPi(100000);
    double et = omp_get_wtime() - ti;
    printf("Pi = %lf time = %lfs\n", pi, et);
    return 0;
}

Overwriting pi1.cpp


## Parallel v2

In [60]:
%%writefile pi2.cpp
#include <stdio.h>
#include <omp.h>

#define PAD 16
#define NUM_THREADS 2

double calcPi(int num_steps) {
    double step;
    int nthreads;
    double pi, sum[NUM_THREADS][PAD];
    step = 1. / (double)num_steps;
    omp_set_num_threads(NUM_THREADS);
#pragma omp parallel
{
    double  x;
    int     nthrds = omp_get_num_threads();
    int     id = omp_get_thread_num();
    if (id == 0) nthreads = nthrds;
    sum[id][0] = 0.0;
    for (int i = id; i < num_steps; i+=nthrds) {
        x = (i + .5) * step;
        sum[id][0] += 4. / (1 + x*x);
    }
}
    for (int i = 0; i < nthreads; ++i) {
        pi += sum[i][0];
    }
    return pi / num_steps;
}

int main() {
    double pi;
    double ti = omp_get_wtime();
    pi = calcPi(100000);
    double et = omp_get_wtime() - ti;
    printf("Pi = %lf time = %lfs\n", pi, et);
    return 0;
}

Overwriting pi2.cpp


## Parallel v3

In [61]:
%%writefile pi3.cpp
#include <stdio.h>
#include <omp.h>

#define PAD 16
#define NUM_THREADS 2

double calcPi(int num_steps) {
    double step;
    int nthreads;
    double pi;
    step = 1. / (double)num_steps;
    omp_set_num_threads(NUM_THREADS);
#pragma omp parallel
{
    double  x;
    int     nthrds = omp_get_num_threads();
    int     id = omp_get_thread_num();
    double  sum = 0.0;
    if (id == 0) nthreads = nthrds;
    for (int i = id; i < num_steps; i+=nthrds) {
        x = (i + .5) * step;
        sum += 4. / (1 + x*x);
    }
#pragma omp critical
    pi += sum;
}

    return pi / num_steps;
}

int main() {
    double pi;
    double ti = omp_get_wtime();
    pi = calcPi(100000);
    double et = omp_get_wtime() - ti;
    printf("Pi = %lf time = %lfs\n", pi, et);
    return 0;
}

Overwriting pi3.cpp


In [73]:
%%writefile pi4.cpp
#include <stdio.h>
#include <omp.h>

double calcPi(int num_steps) {
    double step;
    double x, pi, sum = .0;
    step = 1. / (double)num_steps;
#pragma omp parallel for private(x) reduction(+:sum)
    for (int i = 0; i < num_steps; ++i) {
        x = (i + .5) * step;
        sum += 4. / (1 + x*x);
    }
    pi = sum / num_steps;
    return pi;
}

int main() {
    double pi;
    double ti = omp_get_wtime();
    pi = calcPi(100000);
    double et = omp_get_wtime() - ti;
    printf("Pi = %lf time = %lfs\n", pi, et);
    return 0;
}

Overwriting pi4.cpp


In [74]:
%%script bash
g++ -std=c++17 -fopenmp pi0.cpp -o pi0
g++ -std=c++17 -fopenmp pi1.cpp -o pi1
g++ -std=c++17 -fopenmp pi2.cpp -o pi2
g++ -std=c++17 -fopenmp pi3.cpp -o pi3
g++ -std=c++17 -fopenmp pi4.cpp -o pi4

./pi0
./pi1
./pi2
./pi3
./pi4

Pi = 3.141593 time = 0.001660s
Pi = 3.141593 time = 0.001405s
Pi = 3.141593 time = 0.001356s
Pi = 3.141593 time = 0.001013s
Pi = 3.141593 time = 0.000937s
